In [1]:
import numpy as np

class CNN:
    def __init__(self):
        self.layers = []

    def _apply_conv_layer(self, input_data, num_filters, filter_size, padding='same', activation='relu'):
        input_height, input_width = input_data.shape[0], input_data.shape[1]
        output_height = input_height if padding == 'same' else input_height - filter_size[0] + 1
        output_width = input_width if padding == 'same' else input_width - filter_size[1] + 1
        conv_output = np.zeros((output_height, output_width, num_filters))

        # Loop prin fiecare filtru
        for f in range(num_filters):
            # Loop prin fiecare poziție în imagine
            for h in range(output_height):
                for w in range(output_width):
                    # Extrage fereastra din imagine
                    window = input_data[h:h+filter_size[0], w:w+filter_size[1], :]

                   
                    conv_output[h, w, f] = np.max(window)

        # Aplică activarea
        if activation == 'relu':
            conv_output = np.maximum(conv_output, 0)

        return conv_output  # Returnează rezultatul 

    def _apply_maxpool_layer(self, input_data, pool_size, strides):
        input_height, input_width, input_channels = input_data.shape
        output_height = (input_height - pool_size[0]) // strides[0] + 1
        output_width = (input_width - pool_size[1]) // strides[1] + 1
        pool_output = np.zeros((output_height, output_width, input_channels))

        for h in range(0, input_height - pool_size[0] + 1, strides[0]):
            for w in range(0, input_width - pool_size[1] + 1, strides[1]):
                window = input_data[h:h+pool_size[0], w:w+pool_size[1], :]
                for c in range(input_channels):
                    pool_output[h // strides[0], w // strides[1], c] = np.max(window[:, :, c])

        return pool_output

    def _apply_fc_layer(self, input_data, num_units, activation='relu'):
        input_size = np.prod(input_data.shape)
        input_data_flat = input_data.flatten()
        weights = np.random.randn(input_size, num_units)
        fc_output = np.dot(input_data_flat, weights)

        if activation == 'relu':
            fc_output = np.maximum(fc_output, 0)

        return fc_output
 
    def add_conv_layer(self, num_filters, filter_size, input_channels, padding='same', activation='relu'):
        weights_shape = (*filter_size, input_channels, num_filters)
        weights = np.random.randn(*weights_shape)
        layer = {'type': 'conv', 'num_filters': num_filters, 'filter_size': filter_size, 'input_channels': input_channels, 'padding': padding, 'activation': activation, 'weights': weights}
        self.layers.append(layer)

    def add_maxpool_layer(self, pool_size, strides):
        layer = {'type': 'maxpool', 'pool_size': pool_size, 'strides': strides}
        self.layers.append(layer)

    def add_fc_layer(self, num_units, input_shape=None, weights=None, activation='relu'):
        if weights is None:
            if input_shape is None:
                raise ValueError("Input shape must be provided if weights are not specified.")
            input_size = np.prod(input_shape)
            weights = np.random.randn(input_size, num_units)
        layer = {'type': 'fc', 'num_units': num_units, 'activation': activation, 'weights': weights}
        self.layers.append(layer)

    def _apply_layer(self, layer, input_data):
        if layer['type'] == 'conv':
            return self._apply_conv_layer(input_data, layer['num_filters'], layer['filter_size'], layer['padding'], layer['activation'])
        elif layer['type'] == 'maxpool':
            return self._apply_maxpool_layer(input_data, layer['pool_size'], layer['strides'])
        elif layer['type'] == 'fc':
            return self._apply_fc_layer(input_data, layer['num_units'], layer['activation'])

    def train(self, X_train, y_train, learning_rate=0.001, epochs=10, batch_size=1):
        for epoch in range(epochs):
            for i in range(0, len(X_train), batch_size):
                batch_X = X_train[i:i+batch_size]
                batch_y = y_train[i:i+batch_size]  # Assuming y_train is already in the format [1, 0] or [0, 1]
                batch_y = np.expand_dims(batch_y, axis=(1, 2))  # Add dummy dimensions to match input_data shape



                # Forward pass
                input_data = batch_X
                for layer in self.layers:
                    input_data = self._apply_layer(layer, input_data)
                grad_output = 2 * (input_data - batch_y)  # Gradientul funcției de pierdere (MSE)
                for layer in reversed(self.layers):
                    if layer['type'] == 'conv' or layer['type'] == 'fc':
                        grad_input, grad_weights = self._backward_layer(layer, input_data, grad_output)
                        layer['gradient_weights'] = grad_weights
                        input_data = grad_input  # Actualizăm input_data pentru layer-ul anterior
                        # Actualizarea ponderilor folosind gradient descent
                        if 'weights' in layer:
                            layer['weights'] -= learning_rate * layer['gradient_weights']

            print(f"Epoch {epoch+1}/{epochs}, Loss: ")  # Afișăm loss-ul sau metrica de evaluare la finalul fiecărei epoci

    def evaluate(self, X_test, y_test):
        predictions = self.apply_layers(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
        print(f"Test Accuracy: {accuracy}")

    def apply_layers(self, input_data):
        for layer in self.layers:
            input_data = self._apply_layer(layer, input_data)
        return input_data

    def _backward_layer(self, layer, input_data, grad_output):
        if layer['type'] == 'conv':
            return self._conv_backward(layer, input_data, grad_output)
        elif layer['type'] == 'fc':
            return self._fc_backward(layer, input_data, grad_output)

    def _conv_backward(self, layer, input_data, grad_output):
        grad_input = np.zeros_like(input_data)
        grad_weights = np.zeros_like(layer['weights'])

        for i in range(layer['num_filters']):
            for j in range(grad_output.shape[0]):
                print(layer['weights'][i])
                print(grad_output[i])
                grad_input[j] += np.rot90(layer['weights'][i], 2) @ grad_output[i]
                grad_weights[i] += np.rot90(input_data[j], 2) @ grad_output[i]

        return grad_input, grad_weights

    def _fc_backward(self, layer, input_data, grad_output):
        grad_input = grad_output @ layer['weights'].T
        grad_weights = input_data @ grad_output.T

        return grad_input, grad_weights



In [27]:
import numpy as np
from cmath import exp

class ConvNeuralNetwork():
    def __init__(self) -> None:
        self.filters = []

    def add_conv_layer(self,num_filters: int,filter_size: tuple):
        h,w = filter_size
        filter = [[np.random.randn() for _ in range(w)] for _ in range(h)]
        self.filters.append({'type': 'conv', 'num_filters': num_filters,'size': filter_size, 'filter': filter})
    
    def add_maxpool_layer(self,size:tuple):
        self.filters.append({'type': 'maxpool','size': size})
    
    def add_fully_connected_layer(self,input_size: int, ouput_size: int):
        weights = [[np.random.randn() for _ in range(input_size)] for _ in range(ouput_size)]
        self.filters.append({'type': 'fully connected','weights':weights})
    
    def _apply_conv_layer(self,input, layer: dict):
        width, height, depth = len(input[-1]), len(input), len(input[-1][-1])
        filter = layer['filter']
        filter_size = layer['size']
        num_filters = layer['num_filters']

        start_i = filter_size[0] // 2
        final_i = height - filter_size[0] // 2
        start_j = filter_size[1] // 2
        final_j = width - filter_size[1] // 2

        result = []
        for i in range(start_i, final_i):
            result.append([])
            for j in range(start_j, final_j):
                result[-1].append([])
                for _ in range(num_filters):
                    for k in range(depth):
                        s = 0
                        for h in range(filter_size[0]):
                            for w in range(filter_size[1]):
                                s += filter[h][w]*input[i+h-filter_size[0]//2][j+w-filter_size[1]//2][k]
                        
                        result[i-start_i][j-start_j].append(s)
        
        return result

    def _apply_maxpool_layer(self, input, layer):
        h, w = layer['size']
        width, height, depth = len(input[-1]), len(input), len(input[-1][-1])
        result = []
        for i in range(0, height - h + 1, h):
            result.append([])
            for j in range(0, width - w + 1, w):
                result[-1].append([])
                for k in range(depth):
                    values = []
                    for row in range(i,i+h):
                        for col in range(j,j+w):
                            values.append(input[row][col][k])
                    result[-1][-1].append(max(values))
        
        return result

    def _linear_input(self, input):
        result = []
        width, height, depth = len(input[-1]), len(input), len(input[-1][-1])
        for i in range(height):
            for j in range(width):
                for k in range(depth):
                    result.append(input[i][j][k])
        return result

    def _apply_fully_connected_layer(self, input, layer):
        lin_input = self._linear_input(input)
        weights = layer['weights']
        sigmoid = lambda x : 1 / (1 + exp(-x).real)
        results = []
        for w in weights:
            val = sum([w[i] * lin_input[i] for i in range(len(w))])
            s = sigmoid(val)
            results.append(s)
        return results

    def _apply_layer(self, input, layer):
        type = layer['type']
        if type == 'conv':
            return self._apply_conv_layer(input, layer)
        elif type == 'maxpool':
            return self._apply_maxpool_layer(input, layer)
        elif type == 'fully connected':
            return self._apply_fully_connected_layer(input, layer)

    def _back_propagation_fully_connected_layer(self,layer,errors, input_line):
        learning_rate = 0.00000001
        for i in range(0,len(errors)):
            err = errors[i]
            r = input_line
            for l in self.filters[:-1]:
                r = self._apply_layer(r,l)
            r = self._linear_input(r)
            for j in range(len(r)):
                layer['weights'][i][j] = layer['weights'][i][j] - learning_rate * err * r[j]

    def _back_propagation_conv_layer(self, layer, errors):
        learning_rate = 0.00000001
        for i in range(0, len(errors)):
            filter = layer['filter']
            for k1 in range(len(filter)):
                for k2 in range(len(filter[k1])):
                    layer['filter'][k1][k2] = layer['filter'][k1][k2] - errors[i]*learning_rate
            
    def _back_propagation(self, errors, input_line):
        for layer in reversed(self.filters):
            type = layer['type']
            if type == 'fully connected':
                self._back_propagation_fully_connected_layer(layer, errors, input_line)
            if type == 'conv':
                self._back_propagation_conv_layer(layer,errors)

    def _apply_layers(self, input_line):
        result = input_line
        for layer in self.filters:
            result = self._apply_layer(result, layer)
        return result

    def train(self, input, output, epoch):
        for e in range(epoch):
            print(f"Start epoch {e}")
            for i in range(len(input)):
                input_line = input[i]
                o = output[i]
                result = self._apply_layers(input_line)
                yes_prob = result[0]
                no_prob = result[1]
                if o == 'YES':
                    err = [1-yes_prob, 0-no_prob]
                else:
                    err = [0-yes_prob, 1-no_prob]
                
                self._back_propagation(err,input_line)

    def predict(self,input):
        output = []
        for input_line in input:
            result = self._apply_layers(input_line)
            if result[0] > result[1]:
                output.append('YES')
            else:
                output.append('NO')
        return output


In [28]:
import pandas as pd
from PIL import Image

def read_datas() -> pd.DataFrame:
    df = pd.read_csv('datas.csv')
    df = df.dropna()

    return df

def getTrainingAndValidationDatas():
    np.random.seed(5)
    df = read_datas()
    n = df.shape[0]
    indexes = [i for i in range(n)]
    trainingIndexes = np.random.choice(indexes, int(0.7 * n), replace = False)
    validationIndexes = [i for i in range(n) if not i in trainingIndexes]

    trainingInputs = [df['Photo'].iloc[i] for i in trainingIndexes]
    trainingOutputs = [df['Has Filter'].iloc[i] for i in trainingIndexes]

    validationInputs = [df['Photo'].iloc[i] for i in validationIndexes]
    validationOutputs = [df['Has Filter'].iloc[i] for i in validationIndexes]

    return trainingInputs, trainingOutputs, validationInputs, validationOutputs

def getInputParameters(inputImages, size):
    params = []
    for imagePath in inputImages:
        params.append([])
        image = Image.open(imagePath)
        image = image.resize(size)
        width, height = image.size
        pixel_data = list(image.getdata())
        for y in range(height):
            row_pixels = pixel_data[y * width : (y + 1) * width]
            params[-1].append([])
            for pixel in row_pixels:
                r,g,b = pixel[0],pixel[1],pixel[2]
                maxi = max([r,g,b])
                mini = min([r,g,b])
                if mini == maxi:
                    r = g = b = 0
                else:
                    r = (r-mini)/(maxi-mini)
                    g = (g-mini)/(maxi-mini)
                    b = (b-mini)/(maxi-mini)
                params[-1][-1].append([r,g,b])
    return params

def getClassifier():
    trainingInputs, trainingOutputs, validationInputs, validationOutputs = getTrainingAndValidationDatas()
    x_train = getInputParameters(trainingInputs,(62,62))
    x_valid = getInputParameters(validationInputs,(62,62))
    cnn = ConvNeuralNetwork()
    cnn.add_conv_layer(3,(3,3))
    cnn.add_maxpool_layer((4,4))
    cnn.add_fully_connected_layer(input_size=15*15*9,ouput_size=2)
    cnn.train(x_train, trainingOutputs,10)

    outut = cnn.predict(x_valid)

    for o,vo in zip(outut, validationOutputs):
        print(o,vo)

    return cnn

getClassifier()

Start epoch 0
Start epoch 1
Start epoch 2
Start epoch 3
Start epoch 4
Start epoch 5
Start epoch 6
Start epoch 7
Start epoch 8
Start epoch 9
YES NO
YES NO
NO NO
NO NO
NO NO
NO NO
YES NO
NO NO
YES NO
NO NO
NO NO
NO YES
NO YES
NO YES
NO YES
NO NO
NO YES
YES NO
NO NO
NO NO
YES NO
NO NO
NO NO
NO NO
NO YES
NO YES
NO YES
